In [2]:
import pandas as pd
import math
import numpy as np
import pyodbc
from sqlalchemy import create_engine
import urllib

In [3]:
item_col = 0
desc_col = 1
quant_col = 2
unit_col = 3
rate_col = 4
amount_col = 5

In [31]:
params = urllib.parse.quote_plus('DRIVER={SQL Server};SERVER=WINDOWS-UJU3M60\SQLEXPRESS;DATABASE=SMEC;UID=SMEC;PWD=smecs')
engine = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [32]:
#Read in Excel File to get sheet names
#excel = pd.ExcelFile("Tender Schedule (NEW Branding).xlsm")
excel = pd.ExcelFile(r"C:\Users\User\Documents\UNI\Project\Python\master schedule 221014 numbered.xlsx")
sheets = excel.sheet_names
df = pd.read_excel(r"C:\Users\User\Documents\UNI\Project\Python\master schedule 221014 numbered.xlsx", sheet_name=sheets[0])
df = pd.DataFrame(df, columns = ["Item","Description","Qty","Unit","Rate","Amount"])

Contract_No = df.at[2,'Description']
Project_ID = df.at[2,'Description']
Principal = df.at[1,'Description']
Contractor = df.at[1,'Description']

In [33]:
df_sub_form = pd.DataFrame(columns = ["Primary Key","Project ID","Form ID","Version Number","Item ID","Quantity","Price","Unit","Amount"])

In [34]:
df_forms = pd.DataFrame(columns = {"Form ID", "Form Name"})

In [35]:
#Read pages and convert to Data Frames
#Data Frames stored in DF_List
df_list = list()
inc = 0
for x in sheets :
    df_list.append(pd.read_excel(r"C:\Users\User\Documents\UNI\Project\Python\master schedule 221014 numbered.xlsx", sheet_name=x))
    form = [[inc,x]]
    df_forms = df_forms.append(pd.DataFrame(form,columns = ["Form ID", "Form Name"]),ignore_index = True)
    inc += 1

In [36]:
# df_col = ["Form ID", "Level 1", "Level 2", "Level 3", "Level 4", "Level 5", "Item ID","Description","Units"]
# df = pd.DataFrame(columns = df_col)
# print(df)
print(df_list)

[                          Unnamed: 0  \
0                       Contract No:   
1                           Project:   
2                         Principal:   
3                        Contractor:   
4                                NaN   
5   CONSTRUCTION OF BULK  EARTHWORKS   
6                   PRICING SCHEDULE   
7                                NaN   
8                               Item   
9                                  1   
10                               1.1   
11                               1.2   
12                               1.3   
13                               1.4   
14                               1.5   
15                               1.6   
16                               1.7   
17                               1.8   
18                               1.9   
19                              1.10   
20                              1.11   
21                              1.12   
22                              1.13   
23                              1.14   

In [38]:
rows_list = []
for x in range(0,len(sheets)-1):
    inside = False
    for index, rows in df_list[x].iterrows():
        if inside :
            #Check if at end of table
            if rows[item_col] == "" :
                inside = False
            else :
                hold = ""
                df_hold = ["","0","0","0","0","0","","",""]
                df_colh = ["","","","","","","","",""]
                full_count = 0
                rows[item_col] = str(rows[item_col])
                if rows[item_col] != "nan":
                    count = 0
                    length = len(rows[item_col])
                    for i in str(rows[item_col]):
                        count += 1
                        if i != ".":
                            hold = hold + i
                            if count == length:
                                df_hold[full_count + 1] = hold
                        else:
                            df_hold[full_count + 1] = hold
                            full_count += 1
                            hold = ""
                    df_hold[7] = rows[desc_col]
                    df_hold[0] = df_forms['Form ID'][x]
                    for y in range(0,6):
                        df_hold[6] = str(df_hold[6]) + str(df_hold[y]) + "."
                    df_hold[6] = df_hold[6][:-1]
                    dict1 = {}
                    dict1.update({"Primary Key": "","Project ID":Project_ID,"Form ID":df_forms['Form ID'][x],"Version Number":Version_Number,"Item ID":df_hold[6],"Quantity":rows[quant_col],"Price":rows[rate_col],"Unit":rows[unit_col],"Amount": rows[amount_col]})
                    rows_list.append(dict1)
                    
                    # df_hold[6] = df_hold[6][:-1]
                    # df_colh[1] = Project_ID
                    # df_colh[2] = df_forms['Form ID'][x]
                    # df_colh[3] = Version_Number
                    # df_colh[4] = df_hold[6]
                    # df_colh[5] = rows[quant_col]
                    # df_colh[6] = rows[rate_col]
                    # df_colh[7] = rows[unit_col]
                    # df_colh[8] = rows[amount_col]
                    # df_hold[8] = rows[unit_col]
                    # s_row = pd.Series(df_hold, index=df.columns)
                    # df_row = pd.Series(df_colh)
                    # df_sub_form = df_sub_form.append(df_colh)
                    # df = df.append(s_row,ignore_index=True)
                    #print(df_colh)       

        if rows[item_col] == "Item" :
            inside = True
df =pd.DataFrame(rows_list)
print(df)

    Primary Key  Project ID  Form ID Version Number      Item ID  Quantity  \
0                       NaN        0              3  0.1.0.0.0.0       NaN   
1                       NaN        0              3  0.1.1.0.0.0       NaN   
2                       NaN        0              3  0.1.2.0.0.0       NaN   
3                       NaN        0              3  0.1.3.0.0.0       NaN   
4                       NaN        0              3  0.1.4.0.0.0       NaN   
..          ...         ...      ...            ...          ...       ...   
799                     NaN        4              3  4.4.3.4.0.0       NaN   
800                     NaN        4              3  4.4.3.5.0.0       NaN   
801                     NaN        4              3  4.4.3.6.0.0       NaN   
802                     NaN        4              3  4.4.3.7.0.0       NaN   
803                     NaN        4              3  4.4.3.8.0.0       NaN   

     Price   Unit  Amount  
0      NaN    NaN     NaN  
1      

In [64]:
df.to_sql('Submitted_Forms',engine,if_exists='append')

In [65]:
#df.to_sql('Tender Master', engine, if_exists='replace')

In [66]:
#df_forms.to_sql('Form_IDs', engine, if_exists='replace')

In [67]:
df.to_csv("test_output.csv")

In [68]:
#df_forms.to_csv("Form_IDs.csv")

: 